In [1]:
import logging
from pyomo.environ import *
logging.root.setLevel(logging.NOTSET) # or logging.basicConfig(level=logging.NOTSET)

from config.default_params import create_data_dictionary
from utils.utilities import import_instance

import sys
sys.path.append("../") # go to parent dir

%load_ext autoreload
%autoreload 2

from evrptw import EVRPTW

from utils.graph import draw_plain_graph, draw_graph
from utils.plot import plot_interactive_graph
from utils.utilities import create_optimal_edges, merge_variable_results
from utils.utilities import parse_csv_tables, calculate_distance_matrix, generate_index_mapping
from utils.utilities import trace_routes
import pandas as pd


In [2]:
import json
import numpy as np

# JSON can't handle np.arrays so convert to list
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, tuple):
            return obj.tolist()
        elif isinstance(obj, set):
            return list(obj)
        else:
            return super(NpEncoder, self).default(obj)

In [5]:
def create_json_out(instace_subopt):
    
    data_out = {}
    
    for v in instace_subopt.component_objects(Var):
        data_out[v.name] = {}
        for index in v:
            try:        
                data_out[v.name][str(index)] = value(v[index])
            except:
                data_out[v.name][str(index)] = None

    json_outputs = json.dumps(data_out, cls=NpEncoder, sort_keys=True, indent=4, separators=(',', ': '))

    with open('output.json', 'w') as outfile:
        json.dump(data_out, outfile, cls=NpEncoder, sort_keys=True, indent=4, separators=(',', ': '))
        
    return json_outputs


def update_instance_json(var_list, new_concrete_instance, instance_subopt_json):
    
    for var in var_list:  
        var_instance = getattr(new_concrete_instance, var)  
        for (key, val) in instance_subopt_json[var].items():     
            if var is 'xgamma':
                var_instance[eval(key)] = val 
            else:
                var_instance[key] = val       
        setattr(new_concrete_instance, var, var_instance)
        
    return new_concrete_instance

def update_instance(var_list, new_concrete_instance, instance_subopt):
    
    for var in var_list:
        var_instance = getattr(new_concrete_instance, var)
        for (key, val) in getattr(instance_subopt, var).get_values().items():
            var_instance[key] = val
        
        setattr(new_concrete_instance, var, var_instance)
        
    return new_concrete_instance

In [6]:
instance = 'c106C15'

fpath = 'data/' + instance + '.csv'

m_run = EVRPTW()

duplicates = False
# m_run.solve(fpath, duplicates)

#%% testing initializing instances, solve function

# Specify solver
opt = SolverFactory('gurobi')

# Import data
instance_filepath = fpath
m_run.instance_name = instance_filepath.split('/')[-1].split('.')[0]
logging.info('Importing VRPTW MILP instance: {}'.format(m_run.instance_name))

m_run.data = import_instance(instance_filepath, duplicates)

# Create parameters
logging.info('Creating parameters')
p = create_data_dictionary(m_run.data)

# Create an instance of the AbstractModel passing in parameters
logging.info('Creating instance')
m_run.instance = m_run.m.create_instance(p)

# Solver options
solv_options = {'timelimit': 5} # fix a time limit to create suboptimal solution
    
# Solve instance
logging.info('Solving instance...')
# results = opt.solve(m_run.instance, tee=True)
results = opt.solve(m_run.instance, tee=True, options=solv_options)
logging.info('Done')

INFO:root:Building abstract model
INFO:root:Defining parameters and sets
INFO:root:Defining variables
INFO:root:Defining constraints
INFO:root:Defining objective
INFO:root:Importing VRPTW MILP instance: c106C15
INFO:root:Creating parameters
INFO:root:Creating instance
INFO:root:Solving instance...


    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
Using license file C:\gurobi910\win64\gurobi.lic
Academic license - for non-commercial use only - expires 2021-04-11
Read LP format model from file C:\Users\berwa\AppData\Local\Temp\tmpmy99e15b.pyomo.lp
Reading time = 0.01 seconds
x461: 1108 rows, 403 columns, 4080 nonzeros
Changed value of parameter timelimit to 5.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1108 rows, 403 columns and 4080 nonzeros
Model fingerprint: 0xbfaeebc6
Variable types: 60 continuous, 343 integer (343 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [2e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
P

INFO:root:Done


In [7]:
# Clone the suboptimal solution
instance_subopt1 =  m_run.instance.clone()
instance_subopt2 =  m_run.instance.clone()

# Solve the suboptimal solution to optimality
results = opt.solve(instance_subopt1, tee=True)

Using license file C:\gurobi910\win64\gurobi.lic
Academic license - for non-commercial use only - expires 2021-04-11
Read LP format model from file C:\Users\berwa\AppData\Local\Temp\tmp__nkmsfa.pyomo.lp
Reading time = 0.01 seconds
x461: 1108 rows, 403 columns, 4080 nonzeros
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1108 rows, 403 columns and 4080 nonzeros
Model fingerprint: 0xbfaeebc6
Variable types: 60 continuous, 343 integer (343 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [2e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 402 rows and 109 columns
Presolve time: 0.03s
Presolved: 706 rows, 294 columns, 4674 nonzeros
Variable types: 54 continuous, 240 integer (240 binary)

Root relaxation: objective 2.216407e+02, 94 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objecti

In [8]:
instance_subopt1.obj()

271.2080377727125

In [11]:
#%% testing updating instance variables

var_list = ['xgamma', 'xw', 'xq', 'xa']
m_run.instance = m_run.m.create_instance(p)
m_run.instance = update_instance(var_list, m_run.instance, instance_subopt2)

# m_run.instance.xw.pprint()

    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
xw : Size=22, Index=V01_
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    C10 :     0 :              845.0 :  None : False : False : NonNegativeReals
    C13 :     0 :             1106.0 :  None : False : False : NonNegativeReals
    C18 :     0 :              418.0 :  None : False : False : NonNegativeReals
    C19 :     0 :              513.0 :  None : False : False : NonNegativeReals
    C30 :     0 :  307.3844718719117 :  None : False : False : NonNegativeReals
    C33 :     0 : 407.91726006088516 :  None : False : False : NonNegativeReals
    C35 :     0 :              655.0 :  None : False : False : NonNegativeReals
    C40 :     0 :              199.0 :  None : False : False : NonNegativeReals
    C44 :     0 : 105.99999999999953 :  None : False : False : NonNegati

In [12]:
opt.solve(m_run.instance, tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\berwa\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-09
Read LP format model from file C:\Users\berwa\AppData\Local\Temp\tmpu4ytl1az.pyomo.lp
Reading time = 0.01 seconds
x551: 1350 rows, 487 columns, 5012 nonzeros
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1350 rows, 487 columns and 5012 nonzeros
Model fingerprint: 0x3eb94f6e
Variable types: 66 continuous, 421 integer (421 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 343 rows and 92 columns
Presolve time: 0.04s
Presolved: 1007 rows, 395 columns, 7091 nonzeros
Variable types: 60 continuous, 335 integer (335 binary)
Found heuristic solution: obje

{'Problem': [{'Name': 'x551', 'Lower bound': 371.7001997057637, 'Upper bound': 371.7001997057637, 'Number of objectives': 1, 'Number of constraints': 1350, 'Number of variables': 487, 'Number of binary variables': 421, 'Number of integer variables': 421, 'Number of continuous variables': 66, 'Number of nonzeros': 5012, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '13.845621109008789', 'Error rc': 0, 'Time': 14.19668436050415}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [17]:
#%% creating json output

json_outputs = create_json_out(instance_subopt2)
print(json_outputs)

ERROR:pyomo.core:evaluating object as numeric value: xgamma[D0,D0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[D0,D0]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[D1,D0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[D1,D0]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[D1,D1]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[D1,D1]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[D1,S0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[D1,S0]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[D1,S3]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[D1,S3]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[

ERROR:pyomo.core:evaluating object as numeric value: xgamma[C13,C13]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[C13,C13]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[C10,D0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[C10,D0]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[C10,C10]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[C10,C10]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[C44,D0]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[C44,D0]
ERROR:pyomo.core:evaluating object as numeric value: xgamma[C44,C44]
    (object: <class 'pyomo.core.base.var._GeneralVarData'>)
No value for uninitialized NumericValue object xgamma[C44,C44]
ERROR:pyomo.core:evaluating object as numeri

ERROR: evaluating object as numeric value: xgamma[D0,D0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object xgamma[D0,D0]
ERROR: evaluating object as numeric value: xgamma[D1,D0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object xgamma[D1,D0]
ERROR: evaluating object as numeric value: xgamma[D1,D1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object xgamma[D1,D1]
ERROR: evaluating object as numeric value: xgamma[D1,S0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object xgamma[D1,S0]
ERROR: evaluating object as numeric value: xgamma[D1,S3]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object xgamma[D1,S3]
ERROR: evaluating object as numeric value: xgamma[D1,S7]
        (obje

In [18]:
#%% loading json output

with  open('output.json', 'r') as inputfile:
    json_outputs = json.load(inputfile)

In [27]:
var_list = ['xgamma', 'xw', 'xq', 'xa']
m_run.instance = m_run.m.create_instance(p)
# m_run.instance = update_instance_json(var_list, m_run.instance, json_outputs)
# m_run.instance.xw.pprint()
# m_run.instance.obj()

    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


In [23]:
opt.solve(m_run.instance, tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\berwa\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-09
Read LP format model from file C:\Users\berwa\AppData\Local\Temp\tmp6grhyp22.pyomo.lp
Reading time = 0.01 seconds
x551: 1350 rows, 487 columns, 5012 nonzeros
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1350 rows, 487 columns and 5012 nonzeros
Model fingerprint: 0x3eb94f6e
Variable types: 66 continuous, 421 integer (421 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 343 rows and 92 columns
Presolve time: 0.05s
Presolved: 1007 rows, 395 columns, 7091 nonzeros
Variable types: 60 continuous, 335 integer (335 binary)
Found heuristic solution: obje

{'Problem': [{'Name': 'x551', 'Lower bound': 371.7001997057637, 'Upper bound': 371.7001997057637, 'Number of objectives': 1, 'Number of constraints': 1350, 'Number of variables': 487, 'Number of binary variables': 421, 'Number of integer variables': 421, 'Number of continuous variables': 66, 'Number of nonzeros': 5012, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '17.16280174255371', 'Error rc': 0, 'Time': 17.52383303642273}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}